# Set Up and Loading

In [5]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score, roc_curve
import joblib

# Load the dataset
file_path = './diabetes_risk_prediction_dataset.csv'  # Replace with your file path
data = pd.read_csv(file_path)

# Display basic information
print("=======================================")
print("First 5 Rows of the Dataset:")
print("=======================================")
print(data.head())

print("\n=======================================")
print("Dataset Information:")
print("=======================================")
data.info()

print("\n=======================================")
print("Column Data Types:")
print("=======================================")
print(data.dtypes)

print("\n=======================================")
print("Missing Values Check:")
print("=======================================")
print(data.isnull().sum())

print("\n=======================================")
print("Basic Descriptive Statistics for Numerical Columns:")
print("=======================================")
print(data.describe())

print("\n=======================================")
print("Value Counts for Target Column (Class):")
print("=======================================")
print(data['class'].value_counts())  # Replace 'class' with your actual target column name


First 5 Rows of the Dataset:
   Age Gender Polyuria Polydipsia sudden weight loss weakness Polyphagia  \
0   40   Male       No        Yes                 No      Yes         No   
1   58   Male       No         No                 No      Yes         No   
2   41   Male      Yes         No                 No      Yes        Yes   
3   45   Male       No         No                Yes      Yes        Yes   
4   60   Male      Yes        Yes                Yes      Yes        Yes   

  Genital thrush visual blurring Itching Irritability delayed healing  \
0             No              No     Yes           No             Yes   
1             No             Yes      No           No              No   
2             No              No     Yes           No             Yes   
3            Yes              No     Yes           No             Yes   
4             No             Yes     Yes          Yes             Yes   

  partial paresis muscle stiffness Alopecia Obesity     class  
0          

# 2. Data Preprocessing

In [6]:
# Normalize column names
data.columns = data.columns.str.strip().str.lower().str.replace(' ', '_')
print("Normalized Columns:")
print(data.columns)

# Handle missing values
print("\nMissing Values Check:")
print(data.isnull().sum())  # Check for missing values
data.fillna(method='ffill', inplace=True)  # Forward fill as an example

# Encode categorical variables
label_encoder = LabelEncoder()
categorical_cols = ['gender', 'polyuria', 'polydipsia', 'sudden_weight_loss', 'weakness',
                    'polyphagia', 'genital_thrush', 'visual_blurring', 'itching', 'irritability',
                    'delayed_healing', 'partial_paresis', 'muscle_stiffness', 'alopecia', 'obesity']
for col in categorical_cols:
    if col in data.columns:  # Ensure the column exists
        data[col] = label_encoder.fit_transform(data[col])

# Encode target variable
if 'class' in data.columns:  # Ensure target column exists
    data['class'] = label_encoder.fit_transform(data['class'])


Normalized Columns:
Index(['age', 'gender', 'polyuria', 'polydipsia', 'sudden_weight_loss',
       'weakness', 'polyphagia', 'genital_thrush', 'visual_blurring',
       'itching', 'irritability', 'delayed_healing', 'partial_paresis',
       'muscle_stiffness', 'alopecia', 'obesity', 'class'],
      dtype='object')

Missing Values Check:
age                   0
gender                0
polyuria              0
polydipsia            0
sudden_weight_loss    0
weakness              0
polyphagia            0
genital_thrush        0
visual_blurring       0
itching               0
irritability          0
delayed_healing       0
partial_paresis       0
muscle_stiffness      0
alopecia              0
obesity               0
class                 0
dtype: int64


C:\Users\User\AppData\Local\Temp\ipykernel_14200\3407106011.py:9: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data.fillna(method='ffill', inplace=True)  # Forward fill as an example
